In [ ]:
%pip install transformers nltk scikit-learn

In [146]:
from IPython.display import display, Markdown, Latex
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import movie_reviews
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from transformers import pipeline
import nltk
import pandas as pd
import numpy as np
import re

In [2]:
from sklearn.naive_bayes import (
    BernoulliNB,
    ComplementNB,
    MultinomialNB,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

Загрузим данные.

In [4]:
data = pd.read_csv("./IMDB Dataset.csv")
data.replace(to_replace={"sentiment": {"negative": 0, "positive": 1}}, inplace=True)


In [5]:
rm_quoted = re.compile(r"\".*?\"")

Сделаем версию без наименований в ковычках. Их исключение повышает точность моделей.

In [86]:
names = nltk.corpus.names.words()
cleanedup_data = data.copy(deep=True)
cleanedup_data["review"] = cleanedup_data["review"].apply(
    lambda r: re.sub(rm_quoted, "", r.replace("<br />", "\n"))
)

Также лемматизируем.

In [7]:
arrStopwords = stopwords.words("english")
arrStopwords.extend(["br", "<br>", "<br />"])
lemm_arr = []

wnl = WordNetLemmatizer()

for r in data.iterrows():
    rev = r[1]["review"]

    rev_no_quotes = re.sub(rm_quoted, "", rev)
    rev_tok = word_tokenize(rev_no_quotes)
    rev_tok = [w.lower() for w in rev_tok]

    rev_tok_filtered = [w for w in rev_tok if not w in arrStopwords]
    rev_tok_filtered = [w for w in rev_tok_filtered if w.isalpha()]

    rev_tok_lemmatized = np.array([wnl.lemmatize(w) for w in rev_tok_filtered])

    # reviews_tok_arr.append(rev_tok_filtered)
    lemm_arr.append((rev_tok_lemmatized, r[1]["sentiment"]))

lemm_data = pd.DataFrame(columns=["reviews", "sentiment"], data=lemm_arr)


In [8]:
processing_results = {"raw" : data, "cleaned" : cleanedup_data, "lemmatized" : lemm_data}

## Sentiment analyzer встроенный в NLTK

В NLTK есть предобученный определитель настроения называемый VADER. Можно решить задачу при помощи него.

In [3]:
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\danil\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [10]:
vader = SentimentIntensityAnalyzer()

In [83]:
vader_predictions = []

for r in cleanedup_data.iterrows():
    rev = r[1]["review"]

    vader_predictions.append(
        0 if vader.polarity_scores(rev)["compound"] < 0 else 1
    )

print(f"Точность: {accuracy_score(cleanedup_data['sentiment'], vader_predictions) * 100:.2f}%")

Точность: 69.63%


Уже так можно добится точности в районе 70%. На несколько процентов точность можно повысить, если добавить дополнительные признаки и использовать классификаторы из scikit-learn.

Сначала получим списоки отрицательных и положительных слов из корпуса NLTK.

In [13]:
stopword_en = nltk.corpus.stopwords.words("english")
stopword_en.extend([w.lower() for w in nltk.corpus.names.words()])

def skip_unwanted(pos_tuple):
    word, tag = pos_tuple
    if not word.isalpha() or word in stopword_en:
        return False
    if tag.startswith("NN"):
        return False
    return True

positive_words = [word for word, tag in filter(
    skip_unwanted,
    nltk.pos_tag(nltk.corpus.movie_reviews.words(categories=["pos"]))
)]
negative_words = [word for word, tag in filter(
    skip_unwanted,
    nltk.pos_tag(nltk.corpus.movie_reviews.words(categories=["neg"]))
)]

Потом выберем самые часто встречающиеся положительные и отрицательные леммы.

In [101]:
positive_fd = nltk.FreqDist(positive_words)
negative_fd = nltk.FreqDist(negative_words)

common_set = set(positive_fd).intersection(negative_fd)

for word in common_set:
    del positive_fd[word]
    del negative_fd[word]

top_100_positive_lem = {wnl.lemmatize(w) for w, count in positive_fd.most_common(100)}
top_100_negative_lem = {wnl.lemmatize(w) for w, count in negative_fd.most_common(100)}


Загрузим классифаеры из scikit-learn.

In [102]:
classifiers = {
    "BernoulliNB": BernoulliNB(),
    "ComplementNB": ComplementNB(),
    "MultinomialNB": MultinomialNB(),
    "KNeighborsClassifier": KNeighborsClassifier(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "MLPClassifier": MLPClassifier(max_iter=1000),
    "AdaBoostClassifier": AdaBoostClassifier(),
}

Функция для экстракции признаков. Она собирает количество негативных и позитивных слов, а также значения выдаваемые VADER.

In [103]:
def extract_features(text):
    features = dict()
    pos_word_count = 0
    neg_word_count = 0

    for r in wnl.lemmatize(text):
        for w in r:
            if w in top_100_negative_lem:
                neg_word_count += 1
            elif w in top_100_positive_lem:
                pos_word_count += 1

    vader_scores = vader.polarity_scores(text)
    
    features["compound"] = vader_scores["compound"] + 1
    features["neg"] = vader_scores["neg"]
    features["pos"] = vader_scores["pos"]
    features["pos_word_count"] = pos_word_count
    features["neg_word_count"] = neg_word_count

    return features

Собирём признаки для всего датасета

In [129]:
featurelist = []
feature_names = list(extract_features("I like pickles"))
data_column_names = feature_names + ["sentiment"]

for r in cleanedup_data.iterrows():
    r_features_dict = extract_features(r[1]["review"])
    r_features_dict["sentiment"] = r[1]["sentiment"]
    featurelist.append(r_features_dict)

features_df = pd.DataFrame(data=featurelist)

Используем признаки как вводный массив для разных моделей sklearn. Как видим точность некоторых моделей привосходит точность VADER самого по себе. Теоретически можно попробовать вывести ещё больше значимых признаков из текстов данного датасета и увеличить тем самым точность ещё выше

In [127]:
train_data, test_data = train_test_split(features_df, train_size=0.85)

for name, sklearn_classifier in classifiers.items():
    cls = sklearn_classifier

    x_train = train_data.drop(columns=["sentiment"])
    y_train = train_data["sentiment"]

    x_test = test_data.drop(columns=["sentiment"])
    y_test = test_data["sentiment"]

    model = cls.fit(X=x_train, y=y_train)

    predictions = model.predict(x_test)

    accuracy = accuracy_score(y_test, predictions)
    print(F"{accuracy:.2%} - {name}")

52.91% - BernoulliNB
70.11% - ComplementNB
70.09% - MultinomialNB
69.47% - KNeighborsClassifier
63.65% - DecisionTreeClassifier
70.15% - RandomForestClassifier
72.37% - MLPClassifier
72.49% - AdaBoostClassifier


Главным приемуществом данного метода является его скорость. Обработать весь датасет при помощи VADER можно за секунды

In [145]:
%%timeit -n1 -r1 -o
dummy_predictions = []
for r in cleanedup_data.iterrows():
    rev = r[1]["review"]

    dummy_predictions.append(
        0 if vader.polarity_scores(rev)["compound"] < 0 else 1
    )

1min 33s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 1min 33s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [ ]:
vader_worktime = _

In [167]:
display(Markdown(f"### **Главным приемуществом данного метода является его скорость. Обработать весь датасет при помощи VADER можно за {vader_worktime.average:.2f} секунды**"))

### **Главным приемуществом данного метода является его скорость. Обработать весь датасет при помощи VADER можно за 93.43 секунды**

## Модели с Hugging Face

Также для анализа можно использовать модели предтренированные работоприятные с Hugging Face

In [9]:
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
